# Overview

Let's instruct a dataset from various documents. Here we will use Bonito.The workflow see below:

![](https://cdn.masto.host/sigmoidsocial/media_attachments/files/112/171/384/916/341/941/original/0518bdfdaf362c60.webp)

In [ ]:
%%capture
!pip install -e git+https://github.com/BatsResearch/bonito#egg=bonito

# https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.17.0
!pip install PyMuPDF==1.24.0
!pip install spacy==3.7.4
!pip install huggingface-hub==0.22.1

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

# Loading data

In [ ]:
import fitz

pdf_path='/kaggle/input/pdf-for-data-generation/cssf12_552eng.pdf'

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:  # Iterate through each page
        text += page.get_text()  # Extract text and append it to the text variable
    return text

text = extract_text_from_pdf(pdf_path)  # Call the function with the path to your PDF

# Text to sentences

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Load the English language model

def split_into_sentences(text):
    doc = nlp(text)  # Process the text with SpaCy
    sentences = [sent.text.strip() for sent in doc.sents]  # Extract sentences and strip whitespace
    return sentences

sentences = split_into_sentences(text)  # Split the extracted text into sentences
print(len(sentences))

In [ ]:
print(sentences[500])

# Loading to Huggingface Dataset Format

In [ ]:
from datasets import Dataset

# Assuming sentences is a list of strings, where each string is a sentence
data = {"sentence": sentences}
dataset = Dataset.from_dict(data)
dataset

# Generating the Synthetic Dataset

We are using Bonito library to generate a synthetic dataset for "question generation". However, it also supports a wide array of tasks, see the link in "Acknowledge" section.

In [ ]:
from bonito import Bonito, SamplingParams
from datasets import load_dataset

# Initialize the Bonito model
bonito = Bonito("BatsResearch/bonito-v1")

sampling_params = SamplingParams(max_tokens=256, top_p=0.95, temperature=0.5, n=1)
synthetic_dataset = bonito.generate_tasks(
    dataset,
    context_col="sentence",
    task_type="qg",
    sampling_params=sampling_params
)

In [ ]:
import pandas as pd

df=pd.DataFrame(synthetic_dataset)
df.head()

# Pushing to Hub

In [ ]:
synthetic_dataset.push_to_hub('aisuko/generate_dataset12_552')

# Acknowledge

* https://arxiv.org/pdf/2402.18334.pdf
* https://medium.com/towards-data-science/how-to-generate-instruction-datasets-from-any-documents-for-llm-fine-tuning-abb319a05d91
* https://huggingface.co/BatsResearch/bonito-v1